In [2]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')

# different types of exps use different steps at the last step(3).
recipe = [
    [['ChIP-seq'], ''],
    [['CUT%26RUN'], ''],
]

#Choose the recipe element to run the pipeline on
recipe_no = 1
exp_type, step3 = recipe[recipe_no]

set_url = '/search/?'+ \
          '&'.join(['experiments_in_set.experiment_type='+i for i in exp_type])+ \
          '&type=ExperimentSetReplicate&limit=all' + \
          '&status=pre-release&status=released&status=released%20to%20project'
print(set_url)
#set_url = '/search/?award.project=ENCODE&experimentset_type=replicate&type=ExperimentSetReplicate'

#set_url = '/search/?award.project=4DN&experiments_in_set.biosample.biosource.individual.organism.name=fruit-fly&experimentset_type=replicate&type=ExperimentSetReplicate'
#print set_url

all_sets = ff_utils.search_metadata(set_url , key=my_auth)
counter = 0
completed = 0
completed_acc = []

run_sets = [i for i in all_sets if "CHIP_Pipeline_0.2.5"  not in i.get('completed_processes', [])]

print(len(all_sets), 'total number of sets')
print(len(all_sets)-len(run_sets), 'sets completed')
print(len(run_sets), 'ready for processing')

/search/?experiments_in_set.experiment_type=CUT%26RUN&type=ExperimentSetReplicate&limit=all&status=pre-release&status=released&status=released%20to%20project
51 total number of sets
0 sets completed
51 ready for processing


In [15]:
# sample = [[[b1t1,b1t1],[b1t2,b1t2]],[[b2t1,b2t1],[b2t2,b2t2]]]

my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

def clean_extra(fastq_resp, key):
    if 'extra_files' in fastq_resp:
        ff_utils.patch_metadata({}, obj_id = fastq_resp['uuid'], key = key, add_on = 'delete_fields=extra_files')
        return True
    return False

for a_set in run_sets: 
    paired = ""
    attributions = None
    counter += 1
    replicate_exps = a_set['replicate_exps']
    files = []
    cont_files = []
    obj_key = []
    cont_obj_key = []
    replicate_exps = sorted(replicate_exps, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])
    
    for i in replicate_exps:
        exp = i['replicate_exp']['uuid']
        exp_resp = ff_utils.get_metadata(exp, key = my_auth)
        
        
        #get organism and control
        biosample = exp_resp['biosample']
        organism = list(set([bs['individual']['organism']['name'] for bs in biosample['biosource']]))[0]
        control = ''
        exp_relation = exp_resp.get('experiment_relation')
        if exp_relation:
            rel_type = [i['relationship_type'] for i in exp_relation]
            if 'control for' in rel_type:
                control = "control experiment"
                break
            elif 'controlled by' in rel_type:
                controls = [i['experiment'] for i in exp_relation if i['relationship_type'] == 'controlled by']
                control = controls[0]['uuid']
        else:
            control = False
            break

                    
        # get exp files
        exp_files = exp_resp['files']
        b_index = int(i['bio_rep_no']) - 1
        t_index = int(i['tec_rep_no']) - 1
        for a_file in exp_files:
            f_t = []
            o_t = []
            file_resp = ff_utils.get_metadata(a_file['uuid'], key = my_auth) 
            ex = clean_extra(file_resp, key = my_auth)
            if ex:
                print(file_resp['accession'], 'cleaned extra file')
            # get pair end no
            pair_end = file_resp.get('paired_end')
            if pair_end == '2':
                paired = 'yes'
                continue
            # get paired file
            paired_with = ""
            relations = file_resp.get('related_files')
            if not relations:
                pass
            else:
                for relation in relations:
                    if relation['relationship_type'] == 'paired with':
                        paired = 'yes'
                        paired_with = relation['file']['uuid']
            # decide if data is not paired end reads
            if not paired_with:
                if not paired:
                    paired = 'single'
                else:
                    if paired != 'single':
                        print('inconsistent fastq pair info')
                        continue
                        
                f_t.append(file_resp['uuid'])
                o_t.append(file_resp['display_title'])
            else:
                f2 = ff_utils.get_metadata(paired_with, key = my_auth)
                ex = clean_extra(f2, key = my_auth)
                if ex:
                    print(f2['accession'], 'cleaned extra file')
                f_t.append(file_resp['uuid'])
                o_t.append(file_resp['display_title'])
                f_t.append(f2['uuid'])
                o_t.append(f2['display_title'])
                
            try: 
                files[b_index]
            except:
                files.append([])
            files[b_index].append(f_t)
            files[b_index][t_index] = f_t
            
            try:
                obj_key[b_index]
            except:
                obj_key.append([])
            obj_key[b_index].append(o_t)
            obj_key[b_index][t_index] = o_t
            
    if control:
        control_set_acc = ff_utils.get_metadata(control, key = my_auth)['experiment_sets'][0]['accession']
        cont_set = ff_utils.get_metadata(control_set_acc, key=my_auth)
        cont_replicate_exps = cont_set['replicate_exps']
        cont_replicate_exps = sorted(cont_replicate_exps, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])
    
        for i in cont_replicate_exps:
            cont_exp = i['replicate_exp']['uuid']
            cont_exp_resp = ff_utils.get_metadata(cont_exp, key = my_auth)
            cont_exp_files = cont_exp_resp['files']
            b_index = int(i['bio_rep_no']) - 1
            t_index = int(i['tec_rep_no']) - 1
            for a_file in cont_exp_files:
                f_t = []
                o_t = []
                file_resp = ff_utils.get_metadata(a_file['uuid'], key = my_auth)           
                # get pair end no
                pair_end = file_resp.get('paired_end')
                if pair_end == '2':
                    paired = 'yes'
                    continue
                # get paired file
                paired_with = ""
                relations = file_resp.get('related_files')
                if not relations:
                    pass
                else:
                    for relation in relations:
                        if relation['relationship_type'] == 'paired with':
                            paired = 'yes'
                            paired_with = relation['file']['uuid']
                # decide if data is not paired end reads
                if not paired_with:
                    if not paired:
                        paired = 'single'
                    else:
                        if paired != 'single':
                            print('inconsistent fastq pair info')
                            continue

                    f_t.append(file_resp['uuid'])
                    o_t.append(file_resp['display_title'])
                else:
                    f2 = ff_utils.get_metadata(paired_with, key = my_auth)
                    f_t.append(file_resp['uuid'])
                    o_t.append(file_resp['display_title'])
                    f_t.append(f2['uuid'])
                    o_t.append(f2['display_title'])

                try: 
                    cont_files[b_index]
                except:
                    cont_files.append([])
                cont_files[b_index].append(f_t)
                cont_files[b_index][t_index] = f_t

                try:
                    cont_obj_key[b_index]
                except:
                    cont_obj_key.append([])
                cont_obj_key[b_index].append(o_t)
                cont_obj_key[b_index][t_index] = o_t
              
                
    if not control:
        print('no control exp')
        continue
    
    if organism not in ['mouse', 'human']:
        print('orgamism not ready', organism)
        continue
    
    if paired == 'single':
        print('skipping single end read set')
        continue

                        
    print(a_set['accession'], organism, paired)  
    print(a_set['description'])
    attributions = get_attribution(ff_utils.get_metadata(files[0][0][0], key = my_auth))
    
    caller = ''
    org = ''
    typ = ''
    if organism == "human":
        org = 'hs'
        typ = 'tf'
        caller = 'mac2'
        input_files = [{
              "object_key": "4DNFIZQB369V.bwaIndex.tar",
              "rename": "GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.tar",
              "bucket_name": raw_bucket,
              "workflow_argument_name": "chip.bwa_idx_tar",
              "uuid": "38077b98-3862-45cd-b4be-8e28e9494549"
              },
              {
              "object_key": "4DNFIZ1TGJZR.bed.gz",
              "bucket_name": raw_bucket,
              "workflow_argument_name": "chip.blacklist",
              "uuid": "9562ffbd-9f7a-4bd7-9c10-c335137d8966"
              },
              {
              "object_key": "4DNFIZJB62D1.chrom.sizes",
              "bucket_name": raw_bucket,
              "workflow_argument_name": "chip.chrsz",
              "uuid": "9866d158-da3c-4d9b-96a9-1d59632eabeb"
              }]
          
    elif organism == "mouse":
        org = 'mm'
        typ = 'tf'
        caller = 'mac2'
        input_files = [{
              "object_key": "4DNFIZ2PWCC2.bwaIndex.tar",
              "rename": "mm10_no_alt_analysis_set_ENCODE.fasta.tar",
              "bucket_name": raw_bucket,
              "workflow_argument_name": "chip.bwa_idx_tar",
              "uuid": "f4b63d31-65d8-437f-a76a-6bedbb52ae6f"
              },
              {
              "object_key": "4DNFIZ3FBPK8.bed.gz",
              "bucket_name": raw_bucket,
              "workflow_argument_name": "chip.blacklist",
              "uuid": "a32747a3-8a9e-4a9e-a7a1-4db0e8b65925"
              },
              {
              "object_key": "4DNFIBP173GC.chrom.sizes",
              "bucket_name": raw_bucket,
              "workflow_argument_name": "chip.chrsz",
              "uuid": "be0a9819-d2ce-4422-be4b-234fb1677dd9"
              }]
        
    input_files.append({"object_key": obj_key,
                       "bucket_name": raw_bucket,
                       "workflow_argument_name": "chip.fastqs",
                       "uuid": files})

    input_files.append({"object_key": cont_obj_key,
                       "bucket_name": raw_bucket,
                       "workflow_argument_name": "chip.ctl_fastqs",
                       "uuid": cont_files})
    
    
    
    
    if paired == 'single':
        chip_p = False
    if paired == 'yes':
        chip_p = True
    
    parameters = {
        "chip.qc_report.desc": a_set.get('description'),
        "chip.paired_end": chip_p,
        "chip.bam2ta.regex_grep_v_ta": "chr[MUE]|random|alt",
        "chip.gensz": org,
        "chip.qc_report.name": "CHIP-seq Report",
        "chip.choose_ctl.always_use_pooled_ctl": True,
        "chip.pipeline_type": typ,
        "chip.peak_caller": caller}
    
    wf_info = step_settings('encode-chipseq',organism, attributions)
    tag = '0.2.5'
    run_name = a_set['accession']

    """Creates the trigger json that is used by foufront endpoint.
    """
    input_json = {'input_files': input_files,
                  'output_bucket': out_bucket,
                  'workflow_uuid': wf_info['wf_uuid'],
                  "app_name": wf_info['wf_name'],
                  "wfr_meta": wf_info['wfr_meta'],
                  "parameters": parameters,
                  "config": wf_info['config'],
                  "custom_pf_fields": wf_info['custom_pf_fields'],
                  "_tibanna": {"env": my_env,
                               "run_type": wf_info['wf_name'],
                               "run_id": run_name},
                  "tag": tag
                  }
    run_result =  get_wfr_out(files[0][0][0], 'encode-chipseq 0.2.5', my_auth)
    
    if run_result['status'] == 'running':
        print('wfr is still running')
        continue
    elif run_result['status'].startswith('no'):
        print('starting run')

    r = json.dumps(input_json)
    #print(r)
    e = ff_utils.post_metadata(input_json, 'WorkflowRun/run', key=my_auth)
    url = json.loads(e['input'])['_tibanna']['url']
    display(HTML("<a href='{}' target='_blank'>{}</a>".format(url, e['status'])))
    break
  
    

4DNESC5FAH5U human yes
Replicates of CUT&RUN on K562 cells using H3K4me3 (39159 AM) with a digestion time of 3min
wfr is still running



3 4DNESH4UTRNL DpnII mouse 800  -  4DNEX4KRGMAQ is missing Part2 - Fails at runtaskawsem
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:hi-c-processing-bam_4DNEX4KRGMAQ9b484528-5651-4d72-a271-a9b37a42ab05



In [ ]:
# Move files from opc to pc
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

recipe_no = 1
exp_type, step3 = recipe[recipe_no]
action = False
move_title = 'HiC Processing Pipeline - Preliminary Files'

set_url = '/search/?'+ \
          '&'.join(['experiments_in_set.experiment_type='+i for i in exp_type])+ \
          '&type=ExperimentSetReplicate&limit=all' + \
          '&status=released&status=released%20to%20project'

print(set_url)
set_url = '/search/?award.project=4DN&experimentset_type=replicate&lab.display_title=Chuck+Murry%2C+UW&status=pre-release&type=ExperimentSetReplicate'

# exp
# set_url = '/search/?'+ \
#           '&'.join(['experiment_type='+i for i in exp_type])+ \
#           '&type=Experiment&limit=all' + \
#           '&status=released&status=released%20to%20project'

all_sets = ff_utils.search_metadata(set_url , key=my_auth)

ready_sets_1 = [i for i in all_sets if "HiC_Pipeline_0.2.5" in i.get('completed_processes', [])]
print(len(ready_sets_1))
ready_sets_2 = []
for a_set in ready_sets_1:
    if a_set.get('other_processed_files'):
        print('a')
        print(a_set['accession'])
        if move_title in [i['title'] for i in a_set['other_processed_files']]:
            print('b')
            if a_set.get('processed_files'):
                print('c')
                print('WARN' ,a_set['accession'], 'has items in processed files, skipping ')
                continue
            else:
                ready_sets_2.append(a_set)
print(len(ready_sets_2), 'items are ready')

In [ ]:
# move other processed files to processed files field
action = True
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        if opc:
            print('There are files in processed_files field, expected empty', resp['accession'])
            return False
        else:
            print('it is possible that move already happened, no opc but pc', resp['accession'])
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            if action:
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                # update status of pc to status of set or exp
                release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

        
set_w_apf = 0
exp_w_apf = 0
counter = 0
move_title = 'HiC Processing Pipeline - Preliminary Files'

print(len(ready_sets_2), 'experiment sets in scope')
for a_set in ready_sets_2:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_auth, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_auth)
    if res:
        set_w_apf += 1
        print(set_resp['accession'], 'moved to pc')
  
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_auth, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_auth)
        if res_e:
            exp_w_apf += 1
            print(exp_resp['accession'], 'moved to pc')
    print()

print(set_w_apf)
print(exp_w_apf)